<a href="https://colab.research.google.com/github/jugal-krishna/Object-detection/blob/main/RetinaNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Retinanet_R_50_FPN***

In [ ]:
# Install detectron

!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

In [ ]:
# Import libraries

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import inference_on_dataset, PascalVOCDetectionEvaluator

In [ ]:
# Download datasets

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xvf VOCtrainval_06-Nov-2007.tar
!mv VOCdevkit datasets  

In [ ]:
# Visualize an image

im = cv2.imread("/content/datasets/VOC2007/JPEGImages/000005.jpg")
cv2_imshow(im)

In [ ]:
# Configurations

cfg = get_cfg() 
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = 'MyVOCTraining_retinanet_R_50_FPN_3x'
cfg.DATASETS.TRAIN = ("voc_2007_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml") # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.00025 # pick a good LR
cfg.SOLVER.MAX_ITER = 3000 
cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.RETINANET.NUM_CLASSES = 20

In [ ]:
# Training with the above configurations

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:

%load_ext tensorboard
%tensorboard --logdir MyVOCTraining

In [ ]:

# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
val_dataset = 'voc_2007_val'

# Validation


voc_evaluator = PascalVOCDetectionEvaluator(val_dataset)
val_dataloader = build_detection_test_loader(cfg, val_dataset)
predictor = DefaultPredictor(cfg)
inference_on_dataset(predictor.model, val_dataloader, voc_evaluator)['bbox']['AP50']

In [ ]:
# Visualizing some outputs

from detectron2.utils.visualizer import ColorMode

val_img = DatasetCatalog.get("voc_2007_val")

for d in random.sample(val_img, 7):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format

    v = Visualizer(im[:, :, ::-1],
                   metadata= MetadataCatalog.get('voc_2007_train'), 
                   scale= 1.2
      )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])